In [2]:
import os
import re
import torch
import pandas as pd
from torch import nn
from tqdm import tqdm
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from tokenizers.processors import TemplateProcessing
from trl import SFTConfig, SFTTrainer
from peft import (
    prepare_model_for_kbit_training, 
    LoraConfig, 
    get_peft_model,
    TaskType,
)
from datasets import load_dataset
from accelerate import Accelerator

/opt/conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터셋 정의

In [2]:
repo = 'yjgwak/klue-bert-base-finetuned-squad-kor-v1'
tokenizer = AutoTokenizer.from_pretrained(repo)
dataset = load_dataset("csv", data_files="/home/jovyan/work/prj_data/open/train.csv")
max_length = 512
stride = 128

def preprocess_function(examples):
    questions, contexts, answers = examples["question"], examples["context"], examples["answer"]
    def preprocess_text(text):
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    questions = list(map(preprocess_text, questions))
    contexts = list(map(preprocess_text, contexts))
    answers = list(map(preprocess_text, answers))

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # offset_mapping: [(token1 start, token1 end), (token2 ~, ), ...]
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs['overflow_to_sample_mapping']# inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]

        start_char = contexts[sample_idx].find(answer)
        end_char = start_char + len(answer)
        # sequence_ids: (token=None, question=0, context=1)
        sequence_ids = inputs.sequence_ids(i)

        # 컨텍스트의 시작 및 마지막을 찾는다.
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # 만일 정답이 컨텍스트에 완전히 포함되지 않는다면, 레이블은 (0, 0)임
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # 그렇지 않으면 정답의 시작 및 마지막 인덱스
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
    
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# 데이터 프레임을 전처리합니다
# preprocess_function(dataset["train"][6])# [15281]
train_dataset = dataset["train"].map(
    preprocess_function,
    batched = True,
    remove_columns=dataset["train"].column_names,
)

# roBERTa에서는 삭제, BERT에서는 중요한 역할
# train_dataset = train_dataset.remove_columns("token_type_ids")

In [3]:
idx = 9
sample_idx = train_dataset["overflow_to_sample_mapping"][idx]
answer = dataset['train']['answer'][sample_idx]

start = train_dataset['start_positions'][idx]
end = train_dataset['end_positions'][idx]
labeled_answer = tokenizer.decode(train_dataset["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")
print(tokenizer.decode(train_dataset["input_ids"][idx]))

Theoretical answer: 6세대, labels give: 6세대
[CLS] PM9A3 E1. S가 기반한 V낸드는 몇 세대인가 [SEP] 삼성전자가 OCP ( 오픈 컴퓨트 프로젝트 ) 의 규격을 만족하는 데이터센터 전용 고성능 SSD ‘ PM9A3 E1. S ’ 를 양산한다고 24일 밝혔다. OCP는 글로벌 데이터센터 관련 기업들이 효율적인 데이터센터 개발과 운영에 필요한 하드웨어와 소프트웨어의 표준을 정립하는 기구다. 이번 제품은 업계최초 6세대 V낸드를 기반으로 한 데이터센터 전용 SSD로, OCP의 NVMe Cloud SSD 표준을 지원하며, 데이터센터에서 요구하는 성능, 전력 효율, 보안 등을 각각 최고 수준의 솔루션으로 제공한다. 특히 전력 효율이 업계 최고 수준으로 높아 데이터센터 운영 비용을 절감할 수 있으며, 최근 화두가 되고 있는 탄소 저감 효과도 기대할 수 있다. PM9A3 E1. S의 전력 효율은 연속쓰기 성능을 기준으로 할 때 1와트 ( W ) 당 283MB / s를 지원하며, 이는 이전 세대인 5세대 V낸드 기반 PM983a M. 2 보다 약 50 % 향상됐다. 지난해 전세계 서버용으로 출하된 HDD ( 하드 디스크 드라이브 ) 를 모두 PM9A3 E1. S 4TB로 대체하면 1년간 절감할 수 있는 전력량이 1484GWh에 이른다. PM9A3 E1. S의 연속쓰기 속도는 3000MB / s로 이전 세대인 제품 대비 연속 쓰기 속도가 약 배 증가했으며, 임의읽기 속도 ( 750K IOPS ) 와 임의쓰기 속도 ( 160K IOPS ) 도 각각 40 %, 150 % 향상됐다. 이번 제품은 사용자 데이터 암호화와 같은 기본적인 보안 기능 뿐만 아니라 안티롤백, 보안 부팅 등 다양한 보안 솔루션을 제공한다. 안티롤백은 보안이 취약한 하위 버전의 펌웨어가 다운로드 되지 못하도록 막는 기능으로, PM9A3 E1. S는 보안 취약점이 발견된 펌웨어에 대해서는 이력을 따로 저장해놓고 해당 버전을 다운로드할 경우 정상적으로 처리되지 않도록 막

# 모델 정의

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
print("start")
model = AutoModelForQuestionAnswering.from_pretrained(
        repo,
        device_map="cuda:0",
        torch_dtype=torch.float32,
        quantization_config=quantization_config,
)
print("end")
model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "key", "value"],
    task_type="QUESTION_ANSWERING"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

accelerater = Accelerator()
model, tokenizer = accelerater.prepare(model, tokenizer)

start


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


end


# Loss 정의

# 학습

In [4]:
import wandb
wandb.login()

torch.cuda.empty_cache()
training_args = TrainingArguments(
    output_dir="BERT",
    evaluation_strategy="no",
    num_train_epochs=3,
    save_steps=0.1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=False
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uijinkim. Use `wandb login --relogin` to force relogin
/opt/conda/envs/qa/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [5]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,4.467800
20,3.935200
30,3.912100
40,3.350500
50,3.774900
60,4.312100
70,3.716800
80,3.334300
90,3.356400
100,2.842400


/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoin

TrainOutput(global_step=8991, training_loss=1.051508852569986, metrics={'train_runtime': 9374.7496, 'train_samples_per_second': 15.342, 'train_steps_per_second': 0.959, 'total_flos': 3.797296535699251e+16, 'train_loss': 1.051508852569986, 'epoch': 3.0})

# Inference

In [1]:
import torch
from torch import nn
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from tqdm import tqdm

CHECK_POINT = "/home/jovyan/work/ai_chat_qa_task/code/huggingface/BERT/checkpoint-8991"
TEST_fOLDER = '/home/jovyan/work/prj_data/open/test.csv'
OUTPUT = "test"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
csv = pd.read_csv(TEST_fOLDER)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 모델 및 토크나이저 로드
config = PeftConfig.from_pretrained(CHECK_POINT)
model = AutoModelForQuestionAnswering.from_pretrained(
    config.base_model_name_or_path,
    # quantization_config=quantization_config,
    device_map="cuda:0",
    torch_dtype=torch.float16
)
# model = PeftModel.from_pretrained(model, CHECK_POINT)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# tokenizer.add_tokens(['위챗', '대만 서부', '먀오리현', '(爭訟)'])
# model.resize_token_embeddings(len(tokenizer))

/opt/conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re
TEST_fOLDER = '/home/jovyan/work/prj_data/open/test.csv'
csv = pd.read_csv(TEST_fOLDER)

def get_prediction(question, context):
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        return_tensors="pt",
        truncation="only_second",
        stride=256,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # inputs.pop("token_type_ids")
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    
    with torch.no_grad():
        outputs = model(**inputs)

    # 모든 청크에 대한 start/end 로짓 가져오기
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    
    # 가장 높은 점수의 답변 찾기
    max_answer_score = -float('inf')
    best_answer = ""

    for i in range(len(start_logits)):
        start_indexes = torch.argsort(start_logits[i], descending=True)[:20]
        end_indexes = torch.argsort(end_logits[i], descending=True)[:20]
        
        for start_index in start_indexes:
            for end_index in end_indexes:
                # 답변의 길이를 50 토큰으로 제한
                if end_index < start_index or end_index - start_index + 1 > 50 or end_index - start_index <= 1:
                    continue
                # 답변이 CLS일때는 제외
                if start_index==0 or end_index==0:
                    continue
                
                answer_score = start_logits[i][start_index] + end_logits[i][end_index]
                if answer_score > max_answer_score:
                    max_answer_score = answer_score
                    best_answer = tokenizer.decode(inputs["input_ids"][i][start_index:end_index+1])
                answer = tokenizer.decode(inputs["input_ids"][i][start_index:end_index+1])
    return best_answer
    
submission_dict = {}
for _, row in tqdm(csv.iterrows(), total=len(csv)):
    answer = get_prediction(row['question'], row['context'])
    
    def clean_prediction(text):
        special_tokens = list(tokenizer.special_tokens_map.values())
        pattern = '|'.join(map(re.escape, special_tokens))
        cleaned_text = re.sub(pattern, '', text)
        cleaned_text = ' '.join(cleaned_text.split())
        return cleaned_text

    answer = clean_prediction(answer)
    submission_dict[row['id']] = answer
    print(f"ID: {row['id']} Question: {row['question']} Generated answer: {answer}")
    break
    
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])

  0%|          | 0/1507 [00:00<?, ?it/s]

ID: TEST_0000 Question: 어떤 기관이 지역 중소기업과 소상공인들에게 큰 힘이 되는 통상진흥원인가요 Generated answer: 경제통상진흥원


In [4]:
df.to_csv('bert_stride256.csv', index=False)

In [ ]:
할거: roberta_v2 + answer변경(or 제일 좋았던 모델) -> illuni/illuni-llama-2-ko-7b -> guided text generation